## Training

In [1]:
#!rm -r checkpoints/*

rm: cannot remove 'checkpoints/*': No such file or directory


In [ ]:
# first train with freezed backbone and lr=1e-3
!python train.py \
--snapshot checkpoints/voc_piou/pascal.h5 \
--phi 0 \
--gpu 0,1 \
--no-evaluation \
--random-transform \
--compute-val-loss \
--freeze-backbone \
--batch-size 32 \
--snapshot-path checkpoints/voc_l1 \
pascal /datasets/dataset/VOCdevkit/VOC2012

{'dataset_type': 'pascal', 'detect_quadrangle': False, 'detect_text': False, 'snapshot': 'checkpoints/voc_l1/pascal.h5', 'freeze_backbone': True, 'freeze_bn': False, 'weighted_bifpn': False, 'lr': 0.001, 'batch_size': 32, 'phi': 0, 'gpu': '0,1', 'epochs': 50, 'steps': 10000, 'snapshot_path': 'checkpoints/voc_l1', 'tensorboard_dir': 'logs/2021-02-18', 'snapshots': True, 'evaluation': False, 'random_transform': True, 'compute_val_loss': True, 'multiprocessing': False, 'workers': 1, 'max_queue_size': 10, 'pascal_path': '/datasets/dataset/VOCdevkit/VOC2012'}
Loading model, this may take a second...
Epoch 1/50
361/361 [==============================] - ETA: 0s - loss: 0.5401 - classification_loss: 0.3501 - regression_loss: 0.1900
Epoch 00001: val_loss improved from inf to 0.52528, saving model to checkpoints/voc_l1/pascal.h5
361/361 [==============================] - 683s 2s/step - loss: 0.5401 - classification_loss: 0.3501 - regression_loss: 0.1900 - val_loss: 0.5253 - val_classification_l

In [1]:
# train all layers with lr=1e-4
!python train.py \
--snapshot checkpoints/voc_piou_finetuned/pascal.h5 \
--snapshot-path checkpoints/voc_piou_finetuned \
--phi 0 \
--gpu 0,1 \
--no-evaluation \
--random-transform \
--compute-val-loss \
--lr 6.4e-5 \
--batch-size 16 \
pascal /datasets/dataset/VOCdevkit/VOC2012

{'dataset_type': 'pascal', 'detect_quadrangle': False, 'detect_text': False, 'snapshot': 'checkpoints/voc_l1_finetuned/pascal.h5', 'freeze_backbone': False, 'freeze_bn': False, 'weighted_bifpn': False, 'lr': 6.4e-05, 'batch_size': 16, 'phi': 0, 'gpu': '0,1', 'epochs': 50, 'steps': 10000, 'snapshot_path': 'checkpoints/voc_l1_finetuned', 'tensorboard_dir': 'logs/2021-02-19', 'snapshots': True, 'evaluation': False, 'random_transform': True, 'compute_val_loss': True, 'multiprocessing': False, 'workers': 1, 'max_queue_size': 10, 'pascal_path': '/datasets/dataset/VOCdevkit/VOC2012'}
Loading model, this may take a second...
Epoch 1/50
722/722 [==============================] - ETA: 0s - loss: 0.2214 - classification_loss: 0.1349 - regression_loss: 0.0865
Epoch 00001: val_loss improved from inf to 0.16125, saving model to checkpoints/voc_l1_finetuned/pascal.h5
722/722 [==============================] - 830s 1s/step - loss: 0.2214 - classification_loss: 0.1349 - regression_loss: 0.0865 - val_lo

## Evaluation

In [1]:
from eval.pascal import *
from generators.pascal import PascalVocGenerator
from model import efficientdet
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
phi = 0
weighted_bifpn = False
common_args = {
    'batch_size': 1,
    'phi': phi,
}
test_generator = PascalVocGenerator(
    '/datasets/dataset/VOCdevkit/VOC2012',
    'val',
    shuffle_groups=False,
    skip_truncated=False,
    skip_difficult=True,
    **common_args
)

In [3]:
model_path = 'checkpoints/voc_l1_finetuned/pascal.h5'
input_shape = (test_generator.image_size, test_generator.image_size)
anchors = test_generator.anchors
num_classes = test_generator.num_classes()
model, prediction_model = efficientdet(phi=phi, num_classes=num_classes, weighted_bifpn=weighted_bifpn)
prediction_model.load_weights(model_path, by_name=True)

Instructions for updating:
Use fn_output_signature instead


In [4]:
average_precisions = evaluate(test_generator, prediction_model, visualize=False)

# compute per class average precision
total_instances = []
precisions = []
for label, (average_precision, num_annotations) in average_precisions.items():
    print('{:.0f} instances of class'.format(num_annotations), test_generator.label_to_name(label),
          'with average precision: {:.4f}'.format(average_precision))
    total_instances.append(num_annotations)
    precisions.append(average_precision)
mean_ap = sum(precisions) / sum(x > 0 for x in total_instances)
print('mAP: {:.4f}'.format(mean_ap))

Running network: 100% (5823 of 5823) |###| Elapsed Time: 0:02:49 Time:  0:02:49
Parsing annotations: 100% (5823 of 5823) || Elapsed Time: 0:00:01 Time:  0:00:01


num_fp=395460.0, num_tp=13518.0
433 instances of class aeroplane with average precision: 0.9561
358 instances of class bicycle with average precision: 0.9635
559 instances of class bird with average precision: 0.9555
424 instances of class boat with average precision: 0.8737
630 instances of class bottle with average precision: 0.8846
301 instances of class bus with average precision: 0.9723
1004 instances of class car with average precision: 0.9107
612 instances of class cat with average precision: 0.9930
1176 instances of class chair with average precision: 0.9502
298 instances of class cow with average precision: 0.9754
305 instances of class diningtable with average precision: 0.8373
759 instances of class dog with average precision: 0.9960
360 instances of class horse with average precision: 0.9767
356 instances of class motorbike with average precision: 0.9714
4372 instances of class person with average precision: 0.9471
489 instances of class pottedplant with average precision: 